In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777) 
sample = " If you want you"
idx2char = list(set (sample)) #index -> char
char2idx = {c:i for i , c in enumerate(idx2char)}

In [3]:
dic_size = len(char2idx) #RNN input size
num_classes = len(char2idx)
hidden_size = len(char2idx) #output size
batch_size = 1
sequence_length = len(sample) -1
learning_rate = 0.1

In [4]:
sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]] # 0~n-1
y_data = [sample_idx[1:]] # 1 ~ n

In [5]:
X = tf.placeholder(tf.int32,[None, sequence_length]) #X one-hot
Y = tf.placeholder(tf.int32,[None, sequence_length]) #Y Label

In [6]:
x_one_hot = tf.one_hot(X, num_classes)
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple =True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state =initial_state, dtype = tf.float32)

In [7]:
#FCL
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(outputs, num_classes, activation_fn=None)

In [8]:
#reshape out for squence loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits = outputs, targets = Y, weights= weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict = { X: x_data})
        print( i, "loss: ", l, "prediction: ", result, "true Y: ", y_data)
        
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss:  2.30233 prediction:  [[4 1 1 1 1 1 1 1 1 1 1 1 1 1 1]] true Y:  [[7, 2, 1, 6, 4, 8, 1, 9, 0, 5, 3, 1, 6, 4, 8]]
	Prediction str:  o              
1 loss:  2.13579 prediction:  [[6 1 1 1 1 1 1 1 1 1 1 1 1 1 1]] true Y:  [[7, 2, 1, 6, 4, 8, 1, 9, 0, 5, 3, 1, 6, 4, 8]]
	Prediction str:  y              
2 loss:  1.98594 prediction:  [[6 1 1 6 1 1 1 6 1 1 1 1 6 1 1]] true Y:  [[7, 2, 1, 6, 4, 8, 1, 9, 0, 5, 3, 1, 6, 4, 8]]
	Prediction str:  y  y   y    y  
3 loss:  1.81954 prediction:  [[6 1 1 6 8 8 1 6 1 5 3 1 6 4 8]] true Y:  [[7, 2, 1, 6, 4, 8, 1, 9, 0, 5, 3, 1, 6, 4, 8]]
	Prediction str:  y  yuu y nt you
4 loss:  1.59552 prediction:  [[6 1 1 6 8 8 1 6 1 5 3 1 6 4 8]] true Y:  [[7, 2, 1, 6, 4, 8, 1, 9, 0, 5, 3, 1, 6, 4, 8]]
	Prediction str:  y  yuu y nt you
5 loss:  1.33687 prediction:  [[6 2 1 6 4 8 1 6 3 5 3 1 6 4 8]] true Y:  [[7, 2, 1, 6, 4, 8, 1, 9, 0, 5, 3, 1, 6, 4, 8]]
	Prediction str:  yf you ytnt you
6 loss:  1.05865 prediction:  [[6 2 1 6 4 8 1 6 3 5 3 1 6 4 8]] true Y